In [4]:
import pandas as pd
import pandas_datareader as data 
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import math
import datetime

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

from scipy import stats
from scipy.stats import kurtosis, skew
%matplotlib inline


In [ ]:
start = '2020-01-01'
end = datetime.date.today()

dfeth = data.DataReader('ETH-USD', data_source='yahoo', start= start, end= end)
print(dfeth['Adj Close'])

dfbtc = data.DataReader("BTC-USD", data_source='yahoo', start= start, end= end)
print(dfbtc['Adj Close'])

In [ ]:
plt.plot(dfeth["Adj Close"], dfbtc["Adj Close"], 'o', color = 'green', label = 'Daily Price')

plt.title('ETH vs BTC')
plt.xlabel('ETH')
plt.ylabel('BTC')
plt.legend()
plt.show()

In [ ]:
df = pd.DataFrame({"Price ETH": dfeth["Adj Close"],  "Price BTC": dfbtc["Adj Close"]})
corr = df.corr()
print(corr)

In [50]:
# statistical summary
df.describe()

,Price ETH,Price BTC
count,226.000000,226.000000
mean,215.155934,8801.788155
std,63.988993,1435.994940
min,110.605873,4970.788086
25%,169.967857,7914.483765
50%,213.221344,9155.216309
75%,240.059761,9612.186768
max,428.741791,11878.111328


In [ ]:
# Building the model

Y = dfeth.drop(axis = 1)
X = dbtc.drop(axis = 1)


X_train, X_test, y_train, y_test =train_test_split()
